<a href="https://colab.research.google.com/github/anaguilarar/gee_satellite_data/blob/master/examples/2_Superpixel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
!pip uninstall pyproj
!pip install pyproj
!pip install wget
!pip install geehydro
!pip install geopandas
!pip install rasterio

Uninstalling pyproj-3.0.1:
  Would remove:
    /usr/local/bin/pyproj
    /usr/local/lib/python3.7/dist-packages/pyproj-3.0.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/pyproj.libs/libcurl-9a092e29.so.4.6.0
    /usr/local/lib/python3.7/dist-packages/pyproj.libs/libnghttp2-223fd912.so.14.17.1
    /usr/local/lib/python3.7/dist-packages/pyproj.libs/libproj-27e7e26d.so.19.2.1
    /usr/local/lib/python3.7/dist-packages/pyproj.libs/libsqlite3-69e6c45e.so.0.8.6
    /usr/local/lib/python3.7/dist-packages/pyproj.libs/libtiff-4e10f209.so.5.5.0
    /usr/local/lib/python3.7/dist-packages/pyproj.libs/libz-eb09ad1d.so.1.2.3
    /usr/local/lib/python3.7/dist-packages/pyproj/*
Proceed (y/n)? ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 153, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/uninstall.py", line 79, i

In [2]:
!git clone https://github.com/anaguilarar/gee_satellite_data.git

Cloning into 'gee_satellite_data'...
remote: Enumerating objects: 356, done.
remote: Counting objects: 100% (356/356), done.
remote: Compressing objects: 100% (251/251), done.
remote: Total 356 (delta 208), reused 243 (delta 101), pack-reused 0
Receiving objects: 100% (356/356), 1.45 MiB | 4.57 MiB/s, done.
Resolving deltas: 100% (208/208), done.


In [3]:
import os
os.chdir("gee_satellite_data")

In [4]:
import ee
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=YpyikeDXk8ABqwPvLPG2NReO6wPrQw0s-5bFZ87o0Ds&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5NACXUB5bJ6DxTsmY8XabbhqRfQZ8eP-rRhqaZr_V40SSP-ZbGV2E

Successfully saved authorization token.


In [5]:
import pandas as pd
from scripts import gee_satellite_data
from scripts import gee_functions
from scripts import gis_functions
from scripts import general_functions

## Superpixel: Image Segmentation
In the next section we use the segmetation Simple Non-Iterative Clustering (SNIC) module from GEE (https://developers.google.com/earth-engine/apidocs/ee-algorithms-image-segmentation-snic) to create image segments. 
Each of these segments will be treated as a superpixel. 

We will use each of the superpixels as the input features for the radar images smoothing. we are going trough following steps, to get a radar images smoothing using superpixels from optical images:
 * Apply image segmention to optical images
 * Get polygons from superpixels
 * reduce each radar image using the oplygons


### Sentinel 2A: Surface Reflectance level

Before downloading data, the code internally removes those areas covered by shadows and clouds. 

In [67]:
se2_toa = gee_satellite_data.get_gee_data("2016-01-01",
                                            "2016-09-30", 
                                            "data/col_t3.shp", 
                                            "sentinel2_toa",
                                         cloud_percentage= 85)

As a result, we get a list which contains the images available for quering period. Besides a cover percentage feature is calculted, it reffers to ratio of image clean area respect to our area of interest.

In [68]:
se2_toa.summary

,dates,cover_percentage
0,2016-01-10,91.486280
1,2016-01-30,87.265383
2,2016-02-09,76.143054
3,2016-02-19,99.067342
4,2016-03-10,49.270695
5,2016-04-19,97.825919
6,2016-04-29,98.859293
7,2016-05-09,22.561939
8,2016-05-19,70.230652
9,2016-06-08,0.419447


In [69]:
se2_toa.add_vi_layer("ndvi")

In [171]:
cover_percentage = 96
## filter collection by cover percentage
imgcollection = gee_functions.filtering_bycoverpercentage(se2_toa, cover_percentage)
imgcollection = imgcollection.map(lambda img:
                                  img.unmask(0).clip(se2_toa._ee_sp))
## get image segmentation using SNIC method
snic = gee_functions.gee_snic(imgcollection,se2_toa._ee_sp, bands = ["ndvi"] )
### from image segments to polygons
polygons_fromseg = gee_functions.raster_to_polygons(snic, imgcollection.select(["ndvi"]
                                        ).median().clip(se2_toa._ee_sp), se2_toa._ee_sp, scale = 10)



total images: 18 
total images after cover filter: 9


In [149]:
### in this cell we are going to show the collection median, to visualize the image that will be used for segmentation

truecolorParams = {'gamma': 1.3, 
                   'min': 0,
                   'max': 0.3,
                   'bands': ['B4','B3','B2']
                   }

imagetoplot = imgcollection.median()
gee_satellite_data.plot_eeimage(imagetoplot, truecolorParams, se2_toa.geometry, zoom = 11)

### Sentinel 1 
The instrument onboard the Sentinel-1 satellites transmits in one polarisation (V) and measures in two (V and H), giving the VV and VH bands. The first band in each data file is the signal measured in the same polarisation as the transmission (VV) and the second is measured in the orthogonal polarisation (VH).


In [142]:
s1_imagery = gee_satellite_data.get_gee_data("2016-01-01",
                                            "2016-09-30", 
                                            "data/col_t3.shp", 
                                            "sentinel1", bands = ['VV'])
s1_imagery.summary

,dates,cover_percentage
0,2016-03-26 23:13:23,99.558320
1,2016-05-13 23:13:30,99.559609
2,2016-09-10 23:13:37,99.556874
3,2016-01-14 23:13:28,99.559613
4,2016-01-21 10:42:59,99.558320
5,2016-02-07 23:13:28,99.559613
6,2016-02-14 10:42:59,99.558320
7,2016-03-02 23:13:28,99.558167
8,2016-03-09 10:42:58,99.556873
9,2016-04-02 10:43:00,99.558320


Due to Sentinel 1 mission has two satellites (a and b), it can orbit the earth in a different orientation (Ascending and Descending), the  6 days data can be summarised averaging over a time window. 

In [143]:
s1_imagery.reduce_collection_by_days(10)
s1_imagery._dates_reduced

0    2016-01-21 10:42:59
1    2016-02-07 23:13:28
2    2016-02-14 10:42:59
3    2016-03-02 23:13:28
4    2016-03-09 10:42:58
5    2016-03-30 04:58:12
6    2016-04-19 23:13:29
7    2016-04-26 10:43:00
8    2016-05-17 04:58:16
9    2016-06-06 23:13:32
10   2016-06-13 10:43:03
11   2016-06-30 23:13:33
12   2016-07-07 10:43:05
13   2016-07-28 04:58:20
14   2016-08-17 23:13:36
15   2016-08-24 10:43:07
dtype: datetime64[ns]

In [172]:
s1datasuperpixel= s1_imagery.image_collection.map(lambda image:
                                gee_functions.reduce_image_to_superpixel(image, polygons_fromseg))


Once the superpixels has been obtained, we are going to summary our s1 collection in percentiles

In [173]:
abundance_perc = s1datasuperpixel.reduce(ee.Reducer.percentile([5,10, 25, 50, 75 , 90, 95]))
abundance_perc_orig = s1_imagery.image_collection.reduce(ee.Reducer.percentile([5,10, 25, 50, 75 , 90, 95]))




Finally we compare both results

In [186]:
import folium 
display = ee.Image(0).updateMask(0).paint(polygons_fromseg, '000000', 3);

centergeometry = gis_functions.geometry_center(s1_imagery.geometry)

Map = folium.Map(location=[centergeometry[1],
                               centergeometry[0]], zoom_start=12)

Map.addLayer(ee.Image(abundance_perc_orig).select(['VV_p5', 'VV_p50', 'VV_p95']), {'gamma': 1.3, 'min':-25, 'max':0}, 'orig');
Map.addLayer(ee.Image(abundance_perc).select(['first_p5', 'first_p50', 'first_p95']), {'gamma': 1.3, 'min':-25, 'max':0}, 'optical superpixel');
Map.addLayer(display, {}, 'vectors');


Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)

Map

In [ ]:
## download radar image

In [93]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [189]:
### Download images
import time
image_todownload = ee.Image(abundance_perc)

task = ee.batch.Export.image.toDrive(**{
                'image': ee.Image(image_todownload),
                'description': "s1_percentiles_superpixel",
                'folder': '/content/drive/MyDrive/satellite_images',
                'scale': 10,
                'region': se2_toa._ee_sp['coordinates']
            })

task.start()
while task.active():
    print('Polling for task (id: {}).'.format("radar_percentiles_superpixel"))
    time.sleep(20)

Polling for task (id: test_superpixel).
Polling for task (id: test_superpixel).
Polling for task (id: test_superpixel).
Polling for task (id: test_superpixel).
Polling for task (id: test_superpixel).
Polling for task (id: test_superpixel).
Polling for task (id: test_superpixel).
Polling for task (id: test_superpixel).
Polling for task (id: test_superpixel).
Polling for task (id: test_superpixel).
Polling for task (id: test_superpixel).
Polling for task (id: test_superpixel).
Polling for task (id: test_superpixel).
Polling for task (id: test_superpixel).


In [ ]:
### S2 Metrics

maxvalues = imgcollection.select('ndvi').max()
minvalues = imgcollection.select('ndvi').min()
medianvalues = imgcollection.select('ndvi').median()
stdvalues = imgcollection.select('ndvi').reduce(ee.Reducer.stdDev())
medianvalues = medianvalues.addBands(maxvalues).addBands(minvalues).addBands(stdvalues)
medianvalues.getInfo()